# Lecture x: Response API vs Chat Completions

## Topics to cover:
1. Why use Response API over Chat Completions
2. Streaming Responses
3. Prompt Engineering
4. Function Calling
5. A Simple Agent

It is a never-ending battle of being up-to-date with the latest changes in AI. There will always be changes to APIs and their documentation, new tools, deprecations of previous API endpoints and tools.

Our job as AI Engineers is to make sure we are up-to-date with the relevant documentation of the tools we are using. Usually, providers will give ample notice before any major changes are introduced and old methods are depreciated.

## Why the Responses API
As mentioned in OpenAIs documentation for [Responses vs Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

- Better for agentic workflows
- Stateful API
- Model availability
- Built-in tool use (Like OpenAIs own 'file search' and 'computer use')

## What the basic Chat Completions looked like

In [9]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

In [2]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who is Robert Downey Jr.?"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Robert Downey Jr. is an acclaimed American actor and producer, widely recognized for his versatility and for overcoming significant personal and professional challenges throughout his career. Born on April 4, 1965, in New York City, he is the son of filmmaker Robert Downey Sr.\n\nDowney began acting at a young age, appearing in films such as **"Less Than Zero"** (1987) and gaining critical acclaim for his role as Charlie Chaplin in **"Chaplin"** (1992, earning him an Academy Award nomination). However, his career was marred by struggles with substance abuse and legal issues in the late 1990s and early 2000s.\n\nHe staged a remarkable comeback and is best known in recent years for his portrayal of **Tony Stark/Iron Man** in the Marvel Cinematic Universe, beginning with **"Iron Man"** (2008) and reprising the role in several other Marvel films, which brought him worldwide fame and commercial success. Downey has also starred in other popular movies such as *

In [3]:
completion = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is his most popular movie?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Could you please specify which person or actor you are referring to? That way, I can provide you with the most accurate information about their most popular movie.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


## Responses API

In [ ]:

response = client.responses.create(
  model="gpt-4.1",
  input=[
      {
          "role": "user",
          "content": "Who is Robert Downey Jr.?"
      }
  ]
)

print(response)


In [ ]:

response = client.responses.create(
  model="gpt-4.1",
  previous_response_id=response.id,
  input=[
      {
          "role": "user",
          "content": "What is his most popular movie?"
      }
  ]
)

print(response.output_text)


# Prompt Engineering

### Prompt Elements

A prompt contains any of the following elements:

**Instruction** - a specific task or instruction you want the model to perform

**Context** - external information or additional context that can steer the model to better responses

**Input Data** - the input or question that we are interested to find a response for

**Output Indicator** - the type or format of the output.

In [13]:
from typing import Optional

def get_response(prompt: str, model: str, response_id: Optional[int] = None) -> tuple[str, int]:
    response = client.responses.create(
    model=model,
    previous_response_id=response_id,
    input=[
        {
            "role": "user",
            "content": prompt
        }
    ]
    )

    return response.output_text, response.id

### 1. Zero-shot Prompting

In [14]:
MODEL = "gpt-4o-mini"

In [15]:
PROMPT = """

Classify the following text into one of the following categories:
1. Technology
2. Science
3. Art
4. History

Text: The advancements in quantum computing have the potential to revolutionize the field of cryptography, enabling faster processing and more secure communication methods.

"""

response_text, response_id = get_response(PROMPT, MODEL)
display(Markdown(response_text))

The text fits into the **Technology** category.

### 2. Few-shot/Multi-shot prompting

In [16]:
PROMPT = """

Classify the following text into one of the following categories:

1. Technology
2. Science
3. Art
4. History

Text: The advancements in quantum computing have the potential to revolutionize the field of cryptography, enabling faster processing and more secure communication methods.

EXAMPLES

Q: The invention of the printing press in the 15th century allowed for the mass production of books and the spread of knowledge.
A: Technology

Q: The discovery of penicillin in 1928 marked the beginning of modern antibiotics and has saved countless lives.
A: Science

Q: The Mona Lisa, painted by Leonardo da Vinci in the 16th century, is one of the most famous works of art in history.
A: Art

Q: The fall of the Berlin Wall in 1989 was a significant event in world history, symbolizing the end of the Cold War.
A: History

"""

response_text, response_id = get_response(PROMPT, MODEL)
display(Markdown(response_text))

A: Technology

### 3. Chain-of-Thought (CoT) Prompting

In [26]:
MODEL = "gpt-3.5-turbo"
PROMPT = """

A train leaves City A heading towards City B at 60 miles per hour.
At the same time, another train leaves City B heading towards City A at 40 miles per hour.
The cities are 300 miles apart. However, after 1 hour, the train from City A increases its 
speed by 20 miles per hour, and the train from City B decreases its speed by 10 miles per hour. 
How long after departure will the two trains meet?

"""

response_text, response_id = get_response(PROMPT, MODEL)
display(Markdown(response_text))

To solve this problem, we can set up equations for the distance each train travels until they meet.

Let's denote:
- t as the time (in hours) it takes for the trains to meet after the changes in speed.
- x as the distance the 60 mph train travels at the increased speed of 80 mph.
- y as the distance the 40 mph train travels at the decreased speed of 30 mph.

For the first hour:
- Distance traveled by the 60 mph train = 60 miles
- Distance traveled by the 40 mph train = 40 miles

After the first hour:
- Distance traveled by the 60 mph train = 80t miles
- Distance traveled by the 40 mph train = 30t miles

Since the total distance between the cities is 300 miles, we can create the equation:
80t + 30t = 300
110t = 300
t = 300 / 110
t ≈ 2.73 hours

Therefore, the two trains will meet approximately 2.73 hours after departure.

In [27]:
PROMPT = """

A train leaves City A heading towards City B at 60 miles per hour. 
At the same time, another train leaves City B heading towards City A at 40 miles per hour. 
The cities are 300 miles apart. However, after 1 hour, the train from City A increases its 
speed by 20 miles per hour, and the train from City B decreases its speed by 10 miles per hour. 
How long after departure will the two trains meet?

Think about this problem step by step.

"""

response_text, response_id = get_response(PROMPT, MODEL)
display(Markdown(response_text))

Step 1: Calculate how far each train has traveled after 1 hour.
- The train from City A travels at 60 miles per hour for 1 hour, so it has traveled 60 miles.
- The train from City B travels at 40 miles per hour for 1 hour, so it has traveled 40 miles.

Step 2: Calculate the remaining distance between the two trains after 1 hour.
- The total distance between the two cities is 300 miles.
- After 1 hour, the trains are 60 miles + 40 miles = 100 miles apart.

Step 3: Calculate the relative speed between the two trains after 1 hour.
- The relative speed between the two trains is 60 miles per hour + 40 miles per hour = 100 miles per hour.

Step 4: Determine how long it takes for the trains to cover the remaining distance while moving towards each other at the increased and decreased speeds.
- The train from City A increases its speed by 20 miles per hour, making its new speed 80 miles per hour.
- The train from City B decreases its speed by 10 miles per hour, making its new speed 30 miles per hour.
- The combined speed of the trains moving towards each other is 80 miles per hour + 30 miles per hour = 110 miles per hour.
- The remaining distance between the trains is 200 miles (300 miles - 100 miles).
- To cover 200 miles at 110 miles per hour, it will take 200 miles / 110 miles per hour = 1.82 hours.

Step 5: Calculate the total time taken for the trains to meet.
- The initial 1 hour + the additional 1.82 hours = 2.82 hours.
- Therefore, the two trains will meet 2.82 hours after departure.

Therefore, the two trains will meet approximately 2.82 hours after departure.

### Technique 1: Write clear and concise instructions

In [ ]:
PROMPT = "I need to buy a birthday present for my sister"

response_text, response_id = get_response(PROMPT)
display(Markdown(response_text))

In [ ]:
PROMPT = "I need to buy a birthday present for my sister. She is 12 years old and is interested in ponies. Reply only with 2 options."

response_text, response_id = get_response(PROMPT)
display(Markdown(response_text))

### Technique 2: Split your task into simpler subtasks

In [ ]:
PROMPT = """

Write an article about the importance of muscle for skeletal health and mobility in old age.

Follow these guidelines:
- Jot down high-level bullets on the subject
- Write a short introduction
- Write a detailed section on the importance of muscle for skeletal health
- Write a detailed section on the importance of muscle for mobility
- Write a conclusion summarizing the key points
- Use markdown formatting
- Use a friendly and engaging tone
- Use simple language that is easy to understand
- Use a maximum of 500 words

"""

In [ ]:
response_text, response_id = get_response(PROMPT)
display(Markdown(response_text))

### Technique 3: Allow GPT to think

In [ ]:
PROMPT = """

You will be tasked to build a simple application using Python and Flask.

You MUST plan effectively, thinking through each step before you write code.

I want you to build an app that, upon clicking a button, will generate random Yoda quotes.

# Workflow

## High-level problem solving strategy
1. Deeply understand the user's requirements.
2. Break down the requirements into smaller tasks.
3. For each task, think through the implementation details.
4. Write code for each task, ensuring it works correctly.
5. Test the code to ensure it meets the requirements.

## Code implementation strategy
1. Start with the main application structure.
2. Implement the core functionality.
3. Add any additional features as needed.

## Other notes:
1. The app should look sleep and modern
2. It should feel snappy
3. Do not ask any questions
4. Give me the entire code in one file

"""

In [ ]:
response_text, response_id = get_response(PROMPT)
display(Markdown(response_text))

Let's compare with bad prompt engineering

In [ ]:
PROMPT = "I want you to build an app that, upon clicking a button, will generate random Yoda quotes. The app should be sleek and modern and snappy. Give me all the code in one python file."

response_text, response_id = get_response(PROMPT)
display(Markdown(response_text))